# Introduction to calibration and uncertainty propagation

## Problem to solve

Explain the problem here...

In [ ]:
# pip install the required packages if running in Colab
try:
  import google.colab
  IN_COLAB = True
  %pip install summerepi
except:
  IN_COLAB = False

In [ ]:
# Standard imports, plotting option and constant definition
from typing import List, Union
import pandas as pd
# import plotly.express as px
# import numpy as np
# import pickle

from summer import CompartmentalModel

pd.options.plotting.backend = "plotly"

## Create some dummy data we want our model to fit to

In [ ]:
data = pd.DataFrame({"active_cases":
{
    60.: 3000.,
    80.: 8500.,
    100.: 21000.,
    120.: 40000.,
    140.: 44000.,
    160.: 30000.,
    180.: 16000.,
    200.: 7000.,
}}
)
data['active_cases'].plot(kind="scatter")

# Model

## Define a simple SIR model

In [ ]:
def build_sir_model(parameters: dict) -> CompartmentalModel:
    """
    Create a compartmental model, with the minimal compartmental structure needed to run and produce some sort of 
    meaningful outputs.
    
    Args:
        parameters: Flow parameters
    Returns:
        A compartmental model currently without stratification applied
    """

    model = CompartmentalModel(
        times=(parameters["start_time"], parameters["end_time"]),
        compartments=["S", "I", "R"],
        infectious_compartments=["I"],
    )

    infectious_seed = parameters["infectious_seed"]
    initial_population = parameters["initial_population"]
    assert initial_population >= infectious_seed, "Initial population size must be greater than infectious seed"

    model.set_initial_population(
        distribution=
        {
            "S": initial_population - infectious_seed, 
            "I": infectious_seed
        }
    )
    
    # Susceptible people can get infected
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=parameters["contact_rate"], 
        source="S", 
        dest="I",
    )
    
    # Infectious people recover
    model.add_transition_flow(
        name="recovery",
        fractional_rate=parameters["recovery_rate"],
        source="I",
        dest="R",
    )

    return model

## Run the model with some example parameters

In [ ]:
parameters = {
    "initial_population": 1.e6,
    "infectious_seed": 100.,
    "contact_rate": 0.3,
    "recovery_rate": 1. / 7.,
    "start_time": 0,
    "end_time": 365,
}

# Get an SIR model object
model = build_sir_model(parameters)

# Run the model
model.run()

# Plot the model outputs against the data
output_df = pd.DataFrame({
    "modelled": model.get_outputs_df()["I"],
    "observed": data.active_cases
})
output_df.plot()